In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error
import xgboost as xgb

import json
import os

os.chdir("Test Data/")

random_state = 1

rs1_train = pd.read_csv("Synthetic/rs1_train.csv")
rs1_test = pd.read_csv("Synthetic/rs1_test.csv")
rs2_train = pd.read_csv("Synthetic/rs2_train.csv")
rs2_test = pd.read_csv("Synthetic/rs2_test.csv")
rs3_train = pd.read_csv("Synthetic/rs3_train.csv")
rs3_test = pd.read_csv("Synthetic/rs3_test.csv")
rs4_train = pd.read_csv("Synthetic/rs4_train.csv")
rs4_test = pd.read_csv("Synthetic/rs4_test.csv")
rs5_train = pd.read_csv("Synthetic/rs5_train.csv")
rs5_test = pd.read_csv("Synthetic/rs5_test.csv")


In [ ]:
def test_function(results_df, train, test, model, model_name, dataset_name):
    X_train = train.drop(['y'], axis = 1)
    y_train = train.y
    X_test = test.drop(['y'], axis = 1)
    y_test = test.y
    
    start_time = time.time()
    model.fit(X_train, y_train)
    mse = mean_squared_error(y_test, model.predict(X_test))
    total_time = (time.time()-start_time)
    results_df.loc[len(results_df)] = [dataset_name, model_name, mse, total_time]
    
    return results_df


In [ ]:
import time
def train_hyperparameters(mod, grid, output_name, train):
    start_time = time.time()
    X_train = train.drop(['y'], axis = 1)
    y_train = train.y
    
    cv = KFold(n_splits=5, shuffle=True, random_state=1)

    grid_search = GridSearchCV(estimator=mod, param_grid=grid, 
                               cv=cv, n_jobs=-1, verbose=1, scoring = 'neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    print("Best score:", -1*grid_search.best_score_)
    print("Best params: ", best_params)
    print("Time: ", time.time()-start_time)

    with open('Hyperparameters/Synthetic/'+output_name+'.json', 'w') as f:
        json.dump(best_params, f)

# Random Forest

In [ ]:
param_grid = {
    'n_estimators': [300],
    'max_depth': [12, 15, 18, 21, 24, 27, 30],
    'max_features': ["sqrt", "log2", 1.0, 0.5],
}

rf = RandomForestRegressor(random_state = 1)
train_hyperparameters(rf, param_grid, "rf_rs1", rs1_train)
train_hyperparameters(rf, param_grid, "rf_rs2", rs2_train)
train_hyperparameters(rf, param_grid, "rf_rs3", rs3_train)
train_hyperparameters(rf, param_grid, "rf_rs4", rs4_train)
train_hyperparameters(rf, param_grid, "rf_rs5", rs5_train)


## Testing random forest

In [ ]:
def rf_test_function(results_df, train, test, dataset_name, parameters_name):
    with open('Hyperparameters/Synthetic/' + parameters_name + '.json') as f:
        best_params = json.load(f)
    best_rf = RandomForestRegressor(**best_params, random_state = 1)
    return test_function(results_df, train, test, best_rf, "Random Forest", dataset_name)



In [ ]:
results_df = pd.DataFrame(columns=['Dataset', 'Model', 'Model MSE', 'Time'])
results_df = rf_test_function(results_df, rs1_train, rs1_test, "rs1_train", "rf_rs1")
results_df = rf_test_function(results_df, rs2_train, rs2_test, "rs2_train", "rf_rs2")
results_df = rf_test_function(results_df, rs3_train, rs3_test, "rs3_train", "rf_rs3")
results_df = rf_test_function(results_df, rs4_train, rs4_test, "rs4_train", "rf_rs4")
results_df = rf_test_function(results_df, rs5_train, rs5_test, "rs5_train", "rf_rs5")


In [ ]:
results_df.to_csv("RF Synthetic Results.csv")
results_df


# XGBoost

In [ ]:
param_grid = {
    'max_depth': [4,6,8],
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'reg_lambda':[0.1, 1, 10],
    'n_estimators':[100, 500, 1000],
    'subsample': [0.6, 0.8, 1.0],
}

xgboost = xgb.XGBRegressor(random_state=1)
train_hyperparameters(xgboost, param_grid, "xgb_rs1", rs1_train)
train_hyperparameters(xgboost, param_grid, "xgb_rs2", rs2_train)
train_hyperparameters(xgboost, param_grid, "xgb_rs3", rs3_train)
train_hyperparameters(xgboost, param_grid, "xgb_rs4", rs4_train)
train_hyperparameters(xgboost, param_grid, "xgb_rs5", rs5_train)



In [ ]:
def xgb_test_function(results_df, train, test, dataset_name, parameters_name):
    with open('Hyperparameters/Synthetic/' + parameters_name + '.json') as f:
        best_params = json.load(f)
    best_rf = xgb.XGBRegressor(**best_params, random_state = 1)
    return test_function(results_df, train, test, best_rf, "Xtreme Gradient Boosting", dataset_name)


In [ ]:
results_df = pd.DataFrame(columns=['Dataset', 'Model', 'Model MSE', 'Time'])
results_df = xgb_test_function(results_df, rs1_train, rs1_test, "rs1_train", "xgb_rs1")
results_df = xgb_test_function(results_df, rs2_train, rs2_test, "rs2_train", "xgb_rs2")
results_df = xgb_test_function(results_df, rs3_train, rs3_test, "rs3_train", "xgb_rs3")
results_df = xgb_test_function(results_df, rs4_train, rs4_test, "rs4_test", "xgb_rs4")
results_df = xgb_test_function(results_df, rs5_train, rs5_test, "rs5_test", "xgb_rs5")



In [ ]:
results_df.to_csv("XGB Synthetic Results.csv")
results_df
